# INTRODUCTION

Hi my name is Ning Leung, and I am a Mechanical Engineer looking to explore the world of data science, and I took the Springboard’s “Data Science Career Track”. I was partner up with my mentor Michal, and together with her help, I embarked on the myriads of courses from the curriculum and applied the skillsets onto my capstone projects.
I have spent many years studying math, but programming had never been a major component of my work and statistics has always been a small blimp on my radar. I have had some exposure to writing simple scripts in MATLAB using C and played around in the Neural Network toolkit, but I would never consider myself someone comfortable with the tool or the language. So, as I progress through the coursework, reading textbooks, and watching lecture courses on statistical analysis and using Python for data analytics, it was both a refreshing and humbling experience. The experience definitely reminded me how enjoyable it was to learn something new along the way.
I knew from speaking to different data scientists in the industry, they have pointed me towards the powerful application of regression. When it was time for me to choose my capstone project, Michal had recommended me to explore time-series regression of a multivariate data set. And because I am fascinated by the weather and the environment, I have decided to investigate the pollution and air quality of Beijing. 


# THE PROBLEM

Beijing is famous for its pollution during the winter, and the smog gets especially thicker than usual due to the increased demand for burning coal for heat and electricity. A common way of measuring air quality is to investigate the concentration of fine particulate matter (PM2.5), these matters are a concoction of liquid droplets and small particles created from vehicle exhaust or powerplants with the size being 2.5 microns, hence the name PM2.5.
These small matters could get inside our body and get lodge inside our lungs and travel within the cardiovascular system, needless to say these pollutions can cause health issue. Having a good understanding of the PM2.5 concentration in the air is therefore important. The UCI Machine Learning database contains a set of hourly record of PM2.5 data from 2010 to 2015, measured from the U.S. Embassy at Beijing and it is further enhanced by additional meteorological data from Beijing Capital International Airport. 

So using these information, would it be possible to predict the PM2.5 concentration in the air with respect to time?

# THE SOLUTION

To tackle the problem is simple: we were given a time dependent dataset and looking to forecast into the future by observing the trend exhibited by the data, this type of problem is suitable to use linear regression for analysis. 

Without going into too much detail about linear regression. But a basic google search will yield a trove of examples and explanations on the metholodgies behind Time Series Linear Regression Models. The essential model reovlves around the form:

[Y] = b_0 + b_1 * [x] + e

Where,

b_0 is the intercept for the line represented from the above equation.
b_1 is the slope for the line based on x.
e is the deviation from the line plotted from the model.


In my case, the [x] would be the time variable, while the [y] would be the 2.5PM concentration. The intercept b_0 is where the PM2.5 ([y]) is when time ([x]) is zero.

To move forward with analysing the data, I pulled the csv files into panda dataframe and looked into the data structure:

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
#CSV Pull
df = pd.read_csv('PRSA_data_2010.1.1-2014.12.31.csv', index_col = 0)

In [3]:
print(df.describe())

               year         month           day          hour         pm2.5  \
count  43824.000000  43824.000000  43824.000000  43824.000000  41757.000000   
mean    2012.000000      6.523549     15.727820     11.500000     98.613215   
std        1.413842      3.448572      8.799425      6.922266     92.050387   
min     2010.000000      1.000000      1.000000      0.000000      0.000000   
25%     2011.000000      4.000000      8.000000      5.750000     29.000000   
50%     2012.000000      7.000000     16.000000     11.500000     72.000000   
75%     2013.000000     10.000000     23.000000     17.250000    137.000000   
max     2014.000000     12.000000     31.000000     23.000000    994.000000   

               DEWP          TEMP          PRES           Iws            Is  \
count  43824.000000  43824.000000  43824.000000  43824.000000  43824.000000   
mean       1.817246     12.448521   1016.447654     23.889140      0.052734   
std       14.433440     12.198613     10.268698    

In [4]:
print(df.dtypes)

year       int64
month      int64
day        int64
hour       int64
pm2.5    float64
DEWP       int64
TEMP     float64
PRES     float64
cbwd      object
Iws      float64
Is         int64
Ir         int64
dtype: object


I noticed the [year], [month], [day] and [hour] columns are all in int64 format, I chose to convert these columns into strings to help perform time series analysis and used back fill to imputate an y missing data for the PM2.5 columns. 

In [5]:
df['month']=df.month.apply(str)
df['day']=df.day.apply(str)
df['hour']=df.hour.apply(str)
df['month']=df['month'].apply(lambda x: x.zfill(2))
df['day']=df['day'].apply(lambda x: x.zfill(2))
df['hour']=df['hour'].apply(lambda x: x.zfill(2))
print(df.head())

    year month day hour  pm2.5  DEWP  TEMP    PRES cbwd    Iws  Is  Ir
No                                                                    
1   2010    01  01   00    NaN   -21 -11.0  1021.0   NW   1.79   0   0
2   2010    01  01   01    NaN   -21 -12.0  1020.0   NW   4.92   0   0
3   2010    01  01   02    NaN   -21 -11.0  1019.0   NW   6.71   0   0
4   2010    01  01   03    NaN   -21 -14.0  1019.0   NW   9.84   0   0
5   2010    01  01   04    NaN   -20 -12.0  1018.0   NW  12.97   0   0


In [6]:
# Use back fill to imputate any missing data for PM2.5
# Followed by checking for any NaN remaining
df['pm2.5']=df['pm2.5'].fillna(method='bfill')
df['pm2.5'].isnull().sum()

0

Futher combing the [year], [month], [day], [hour] columns into a single column calle [period], now I can plot the PM2.5 vs. Time for regression analysis.

In [7]:
df["period"] = df["year"].map(str) + '-' + df["month"].map(str) + '-' + df["day"].map(str) + '-' + df["hour"].map(str) + ':00:00'
print(df["period"].head())
print(df["period"].tail())

No
1    2010-01-01-00:00:00
2    2010-01-01-01:00:00
3    2010-01-01-02:00:00
4    2010-01-01-03:00:00
5    2010-01-01-04:00:00
Name: period, dtype: object
No
43820    2014-12-31-19:00:00
43821    2014-12-31-20:00:00
43822    2014-12-31-21:00:00
43823    2014-12-31-22:00:00
43824    2014-12-31-23:00:00
Name: period, dtype: object


Plotting the PM2.5 column agains the period column, this is what returns:

In [8]:
plt.plot(df['period'],df['pm2.5'], label='pm2.5')
plt.yscale('linear')
plt.legend(loc='upper right')
plt.title('PM2.5 between 2010-2014')
plt.show()

ValueError: could not convert string to float: '2014-12-31-23:00:00'

The plots showed the dataset possess seasonality - the concentration of the PM2.5 particulate matters are higher during the winter periods of November to February than the summer months of May to September, and the cyclic trend can be observed annually between the individual year plots. 

To perform a regression analysis, I used Facebook Prophet, a helpful opensource tool created specifically to forecast time series data at scale.

For additional information, you can visit the site here: https://facebook.github.io/prophet/

# FACEBOOK PROPHET

In [9]:
!pip install fbprophet
from fbprophet import Prophet
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
D:\Anaconda\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Creating a new "period" column for dates

To check predicted values versus the actuaL PM2.5 values from Prophet's forecasting, the results from the year 2014 was omitted from the training data set, and a new 'period' time series with data ranging from 2010 to 2013. 

In [10]:
#Isolated 2014 for prediction later
df.loc[df['year'] != 2014];
df["period"] = df["year"][:35063].map(str) + '-' + df["month"].map(str) + '-' + df["day"].map(str) + '-' + df["hour"].map(str) + ':00:00'
print(df["period"].tail())

No
43820    NaN
43821    NaN
43822    NaN
43823    NaN
43824    NaN
Name: period, dtype: object


In [11]:
# Check for any NaN in the Dew Point Column
print(df['DEWP'].isnull().sum())
dewp = df['DEWP'];

0


# First Forecast

In [12]:
# Creating a new DataFrame for time and PM2.5 data between 2010 and 2013.
df0 = df[['period', 'pm2.5']].fillna(method='bfill')
df0=df0.dropna()
df0.columns = ['ds','y']
print(df0.tail())

                        ds     y
No                              
35059  2013-12-31-18:00:00  15.0
35060  2013-12-31-19:00:00  22.0
35061  2013-12-31-20:00:00  18.0
35062  2013-12-31-21:00:00  23.0
35063  2013-12-31-22:00:00  20.0


In [13]:
m = Prophet()
m.fit(df0);
future = m.make_future_dataframe(periods=8761, freq='H')
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
forecast.shape

(43824, 22)

In [14]:
m.plot(forecast);

In [15]:
forecast_components = m.plot_components(forecast)

# First Forecast - Results
Notice the forecast of the PM2.5 values into the 2014 as well as the seasonality between the daily, weekly, and monthly values.

From the first prediction using time as a sole regressor, we see:
    
    1. The PM2.5 values are higher during the evening and lower during the day time.
    2. The PM2.5 is peaks during wednesday and weekends.
    3. The winter months have a higher concentration of matter than the summer months.
    
But this is only the beginning of the Time-Series forecasting, Prophet allow for multiple regressions into the "ds" dataset. Now I will add dewpoint "DEW" into the dataframe.

# Second Forecast - 1 Additional Regressor

In [16]:
df1 = df[['period', 'pm2.5', 'DEWP']].fillna(method='bfill')
df1=df1.dropna()
print(df1.head())
print(df1.tail())

                 period  pm2.5  DEWP
No                                  
1   2010-01-01-00:00:00  129.0   -21
2   2010-01-01-01:00:00  129.0   -21
3   2010-01-01-02:00:00  129.0   -21
4   2010-01-01-03:00:00  129.0   -21
5   2010-01-01-04:00:00  129.0   -20
                    period  pm2.5  DEWP
No                                     
35059  2013-12-31-18:00:00   15.0   -19
35060  2013-12-31-19:00:00   22.0   -19
35061  2013-12-31-20:00:00   18.0   -21
35062  2013-12-31-21:00:00   23.0   -21
35063  2013-12-31-22:00:00   20.0   -21


In [17]:
df1.columns = ['ds','y', 'DEWP']

### Using [period], and [DEWP] as Additional Regressors in Prophet

In [18]:
# Calling the FB Prophet method, adding a second regressor for DEWP
m = Prophet()
m.add_regressor('DEWP')
m.fit(df1);

In [ ]:
# The prediction period was one year in hours, hence the period is 8761 points. 
# Appending the DEWP colunms into the future dataframe, including DEWP data for 2014.
future1 = m.make_future_dataframe(periods=8761, freq='H')
future1.loc[:,'DEWP']=df['DEWP']
future1 = future1.iloc[1:]
future1.isnull()

,ds,DEWP
1,False,False
2,False,False
3,False,False
4,False,False
5,False,False
6,False,False
7,False,False
8,False,False
9,False,False
10,False,False


In [ ]:
forecast1 = m.predict(future1)
forecast1[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
forecast1.shape

In [ ]:
forecast1_plot = m.plot(forecast1)

In [ ]:
forecast1_components = m.plot_components(forecast1)

# Second Forecast - Results

Compared to the First Forecast, the forecast1 plot looked to have closer resemblance to the actual measured data. Although it is worthy to point that the predicted values goes into negative, which we know that to be false. However, looking into the seasonality of the data, I see the following: 
    
    1. Again, the results of the PM2.5 values are higher during the evening and lower during the day time.
    2. The PM2.5 concentrate is higher during mid week and Sunday, in particular.
    3. The seasonality over the year shows that winter is again the period with higher concentration of PM2.5 values.
    4. The trend is increasing with an additional regression in the Second Forecast, which is different from the negative slope from the First Forecast.
    
Next, I analyzed the data set with two regressors, Temperature and Dew Point, to see how the prediction changes.

# Third Forecast - 2 Additional Regressors

In [ ]:
print(df['TEMP'].isnull().sum())
temp = df['TEMP']
df2 = df[['period', 'pm2.5', 'DEWP', 'TEMP']].fillna(method='bfill')
df2=df2.dropna()

In [ ]:
df2.columns = ['ds', 'y', 'DEWP', 'TEMP']
print(df2.head());
# Calling the FB Prophet method, adding a second regressor for TEMP
m = Prophet()
m.add_regressor('DEWP')
m.add_regressor('TEMP')
m.fit(df2);

### Using [period], [DEWP] and [TEMP] as Additional Regressors in Prophet

In [ ]:
# The prediction period was one year in hours, hence the period is 8761 points. 
# Appending the DEWP colunms into the future dataframe, including DEWP data for 2014.
future2 = m.make_future_dataframe(periods=8761, freq='H')
future2.loc[:,'DEWP']=df['DEWP']
future2.loc[:,'TEMP']=df['TEMP']
#################################
# For some reason, index row 0 contains a NaN that cannot be removed, the DataFrame was reset by starting off at index row 1.
#################################
future2 = future2.iloc[1:]
future2.isnull()

In [ ]:
forecast2 = m.predict(future2);

In [ ]:
forecast2_plot = m.plot(forecast2)

In [ ]:
forecast2_components = m.plot_components(forecast2)

In [ ]:
forecast2[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
forecast2.shape;

# Third Forecast - Results

The third forecast with temperature and dew points as additional regressors created similar results with the second trial, the data showed the following:
    
    1. PM2.5 values are higher during the evening.
    2. Midweek and sundays have higher concentration.
    3. Winter concentration > Summer concentration.
    4. The trend also has a positive slope.
    
To compared the results by numbers, I looked into the R squared values of the predicted values [yhats] to actual values [pm2.5]

### Comparing Predicted Data vs. Actual using RMSE and MAE

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

In [ ]:
# Mean Squared Error
mse = mean_squared_error(df['pm2.5'][35063:], forecast['yhat'][35063:])
mse1 = mean_squared_error(df['pm2.5'][35064:], forecast1['yhat'][35063:])
mse2 = mean_squared_error(df['pm2.5'][35064:], forecast2['yhat'][35063:])
print(mse)
print(mse1)
print(mse2)

In [ ]:
# Calculate RMSE
rmse = sqrt(mse)
rmse1 = sqrt(mse1)
rmse2 = sqrt(mse2)
print('First Forecast RMSE is ' +str(rmse))
print('Second Forecast RMSE is ' +str(rmse1))
print('Third Forecast RMSE is ' +str(rmse2))

In [ ]:
# Calculate the Mean Absolute Errors
print(mean_absolute_error(df['pm2.5'][35063:], forecast['yhat'][35063:]))
print(mean_absolute_error(df['pm2.5'][35064:], forecast2['yhat'][35063:]))

# Results Summary

The major take away was that the RMSE for the additional regressors decreased, the the errors are minimized. Which show that adding the additional regressors help improve the accuracy of the results. However, the change between the second and third forecast with the additional [TEMP] regressors only marginal improvement, which could limit the prediction by overfitting.

Additional sampling could be done by additional a 3rd regressor to the data set, as well as switching the second forecasting with [TEMP] instead of [DEWP] to compare the difference in values. 

To compare results, I'm going to use another time-series regression modeling with AutoRegressive Integrated Moving Average (ARIMA) and see if I could produce similar forecasting.

## ARIMA MODELING



In [ ]:
year_2014 = df["year"][35064:].map(str) + '-' + df["month"].map(str) + '-' + df["day"].map(str) + '-' + df["hour"].map(str) + ':00:00'
year_2014 = year_2014.dropna()
pm25 = df['pm2.5'][35064:]
data = pd.concat([year_2014, pm25], axis=1, keys=['year_2014','pm25'])
data.set_index('year_2014', inplace=True)
data.index = pd.to_datetime(data.index)

### ACF & PACF Plots

In [ ]:
lag_acf = acf(data, nlags=365)
lag_pacf = pacf(data, nlags=10, method='ols')

In [ ]:
# plot acf
plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.title('Autocorrelation Function')

In [ ]:
# plot pacf
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.title('Partial Autocorrelation Function')

### AR

In [ ]:
model = ARIMA(data, order=(15,1,0))
results_AR = model.fit(disp=-1)
plt.plot(data)
plt.plot(results_AR.fittedvalues, color='red')
plt.xlim('2014-01-01', '2014-12-31')

In [ ]:
np.asarray(data)
data.dtypes
data.tail()

In [ ]:
model = ARIMA(data, order=(0,1,2))
results_MA = model.fit(disp=-1)
plt.plot(data)
plt.plot(results_MA.fittedvalues, color='red')
plt.xlim('2014-01-01', '2014-12-31')

In [ ]:
model = ARIMA(data, order=(2,1,2))
results_ARIMA = model.fit(disp=-1)
plt.plot(data)
plt.plot(results_ARIMA.fittedvalues, color='red')
plt.xlim('2014-01-01', '2014-12-31')

# CONCLUSION

prophet can give you a daily, weekly, monthly and annual data.